In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect("TESTE_GLOBO_DB.db")

In [3]:
cursor = connection.cursor()

### 1. O time de negócios precisa tomar uma decisão de quais títulos licenciar e publicar na plataforma no proximo ano, mas para isso eles precisam saber quais os genero mais assistidos.


In [7]:
sql_1 = '''
    SELECT 
        genero, 
        SUM(horas_assistidas) 
    FROM dts_channel_analytics 
    GROUP BY genero 
    ORDER BY 2 DESC;
'''

In [8]:
resp1 = cursor.execute(sql_1)

In [9]:
resp1.fetchall()[0:3]

[('Romance', 260749), ('Mistery', 255284), ('Humor', 254615)]

Com essa consulta acima conseguimos ver os 3 gêneros mais assistidos. Podemos remover o [0:3] dali afim de consultar tudo, decidi limitar na visualização ao invés da consulta.

### 2. O time de marketing quer a lista dos 10 títulos mais assistidos para realizar uma nova campanha de engajamento

In [10]:
sql_2 = '''
    SELECT 
        titulo, 
        SUM(horas_assistidas) 
    FROM dts_channel_analytics 
    GROUP BY titulo 
    ORDER BY 2 DESC
    LIMIT 10;
'''

In [11]:
resp2 = cursor.execute(sql_2)

In [12]:
resp2.fetchall()

[('\xa0Farmers Of The Mountains', 21623),
 ('\xa0Scent Of The Sea', 19674),
 ('\xa0Fate Of Shadows', 19468),
 ('\xa0Loss', 19458),
 ('\xa0Wives And Rebels', 19170),
 ('\xa0Rat Of The Eternal', 19079),
 ('\xa0Agents Of Heaven', 18933),
 ('\xa0 And Dogs', 18638),
 ('\xa0Rescue In Nature', 17966),
 ('\xa0Attracted To My Future', 17564)]

Com a consulta acima podemos verificar os 10 gêneros mais assistidos, baseado em horas de streaming.

### 3. O time de tecnologia está querendo fazer uma otimização na infraestrutura da plataforma e precisa saber qual foi o data e hora com maior número de usuários assistindo simultaneamente e quantos usuários estavam assistindo neste momento


In [20]:
sql_3 = '''
    SELECT 
        data, 
        hora, 
        sum(usuarios_distintos) 
    FROM dts_channel_analytics 
    GROUP BY data, hora 
    ORDER BY 3 desc 
    LIMIT 1;
'''

In [21]:
resp_3 = cursor.execute(sql_3)

In [22]:
resp_3.fetchall()

[('2021-11-18 00:00:00', 6, 2146)]

Podemos ver com a consulta acima que no dia 18 de novembro de 2021, na hora 6, tivemos 2146 usuários no total, nas 3 plataformas possíveis.

### 4. Sugira uma possível melhoria na base de dados e explique


Não tenho total conhecimento do negócio, mas julgo que talvez fosse mais interessante separar a parte que remete aos dados de horas assistidas e aonde foram assistidas, as qual compreendo como informação não trackeável por marcas de tempo e sim por quantidade de tempo.
Isso talvez facilite a leitura futura de outros analistas afim de obter informação sobre mídia consumida.

O mesmo se estende para o outro lado da moeda, aonde temos dados de data e também de hora, que apesar de também representarem dados e com certeza se trabalhados nos trazem informação, sendo observados juntamente com os dados de horas assistidas, podem causar certa confusão - pois foi o que senti ao observar rapidamente a massa de dados.

### 5. Supondo que a base seja atualizada todo dia com novos dados, que as consultas anteriores que você fez atenderam super bem os times e que agora eles querem receber essas informações diariamente. Descreva qual estratégia você faria para atendê-los.


A estratégia de processamento diário pode ser feita usando uma esteira de processamento clássica(se posso dizer assim) de dados.
Uma instância do Airflow com algumas DAGs que realizem as extrações dos dados diários, após isso uma DAG que processe efetivamente o modelo que gera esse dataset, e por fim outra DAG que processe e popule algum dashboard ou envie as informações por Excel para os stakeholders em questão.

Essa é uma estratégia clássica que não se aprofunda em absolutamente nada na parte técnica da implementação, e sim observa o que é feito no mercado e traz para a necessidade - afinal uma vez que existam tais necessidades de carregamento diário destes dados, no futuro possam haver novas.

Contudo caso não haja budget para implementação de tais coisas, afim de resolver em tempo possa ser melhor apenas utilizar cronjobs que rodem códigos simples que executem essas necessidades, numa VM simples. Mas também não me aprofundarei na parte técnica da implementação, por falta de noção de negócio e de base 